In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel('test.xlsx')
data.head(5)

,Rank,County,DSHS nursing home beds,Total Population,Male,Female,White Total,White Male,White Female,Black Total,...,85+ Asian Female,85+ NHOPI Total,85+ NHOPI Male,85+ NHOPI Female,85+ Hispanic Total,85+ Hispanic Male,85+ Hispanic Female,85+ Two or More Races Total,85+ Two or More Races Male,85+ Two or More Races Female
0,31,Adams County,39,20150,10233.52,9916.48,18056,9185.27,8870.73,446,...,1.32,0.0,0.0,0.0,25.03,12.16,12.86,1.30,1.30,0.00
1,28,Asotin County,90,22520,11100.48,11419.52,21087,10434.52,10652.48,144,...,2.02,2.1,0.0,2.1,11.97,6.23,5.74,6.55,6.55,0.00
2,10,Benton County,364,201800,101396.07,100403.93,181938,91546.48,90391.52,3378,...,36.25,0.0,0.0,0.0,114.56,63.50,51.06,61.50,8.08,53.42
3,17,Chelan County,240,78420,39199.90,39220.10,73391,36703.57,36687.43,510,...,3.51,0.0,0.0,0.0,79.82,39.29,40.53,16.54,4.03,12.51
4,18,Clallam County,303,76010,37927.37,38082.63,65939,32773.48,33165.52,791,...,16.03,0.0,0.0,0.0,27.72,10.13,17.60,22.90,13.03,9.87


In [4]:
y = data['DSHS nursing home beds']
y.head(5)

0     39
1     90
2    364
3    240
4    303
Name: DSHS nursing home beds, dtype: int64

In [5]:
x = data.drop(['County','DSHS nursing home beds', 'Rank'],axis=1)
x.head(5)

,Total Population,Male,Female,White Total,White Male,White Female,Black Total,Black Male,Black Female,AIAN Total,...,85+ Asian Female,85+ NHOPI Total,85+ NHOPI Male,85+ NHOPI Female,85+ Hispanic Total,85+ Hispanic Male,85+ Hispanic Female,85+ Two or More Races Total,85+ Two or More Races Male,85+ Two or More Races Female
0,20150,10233.52,9916.48,18056,9185.27,8870.73,446,265.57,180.43,1109,...,1.32,0.0,0.0,0.0,25.03,12.16,12.86,1.30,1.30,0.00
1,22520,11100.48,11419.52,21087,10434.52,10652.48,144,81.98,62.02,356,...,2.02,2.1,0.0,2.1,11.97,6.23,5.74,6.55,6.55,0.00
2,201800,101396.07,100403.93,181938,91546.48,90391.52,3378,1845.42,1532.58,2504,...,36.25,0.0,0.0,0.0,114.56,63.50,51.06,61.50,8.08,53.42
3,78420,39199.90,39220.10,73391,36703.57,36687.43,510,300.71,209.29,1497,...,3.51,0.0,0.0,0.0,79.82,39.29,40.53,16.54,4.03,12.51
4,76010,37927.37,38082.63,65939,32773.48,33165.52,791,571.02,219.98,4242,...,16.03,0.0,0.0,0.0,27.72,10.13,17.60,22.90,13.03,9.87


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.5, random_state = 45)

In [7]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
rf.fit(x_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(x_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(x_valid, y_valid)))

R^2 Training Score: 0.98 
OOB Score: 0.82 
R^2 Validation Score: 0.12


In [8]:
rf.score(x_valid, y_valid)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

                                  importance
65-69 Two or More Races Total   2.752110e-02
65-69 Female                    2.748072e-02
65-69 Two or More Races Female  2.745495e-02
80-84 NHOPI Female              2.683779e-02
65-69 AIAN Male                 2.294075e-02
...                                      ...
Senior AIAN Female              9.069497e-07
85+ Total                       5.019004e-07
NHOPI Total                     4.896037e-07
75-79 Black Male                2.314885e-07
85+ Asian Male                  0.000000e+00

[168 rows x 1 columns]


In [9]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [10]:
x_train.shape, x_valid.shape

((20, 168), (20, 168))

In [11]:
scaler = StandardScaler()
scaler.fit(x_train.fillna(0))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_.fit(scaler.transform(x_train.fillna(0)), y_train)

C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Leo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [13]:
sel_.get_support()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [14]:
selected_feat = x_train.columns[(sel_.get_support())]
print('total features: {}'.format((x_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 168
selected features: 12
features with coefficients shrank to zero: 2504


In [15]:
np.sum(sel_.estimator_.coef_ == 0)

2504

In [16]:
x_train_selected = sel_.transform(x_train.fillna(0))
x_valid_selected = sel_.transform(x_valid.fillna(0))
x_train_selected.shape, x_valid_selected.shape

((20, 12), (20, 12))

In [17]:
print(selected_feat)


Index(['65-69 Asian Male', '70-74 White Male', '70-74 Hispanic Male',
       '80-84 Black Female', '80-84 AIAN Male', '80-84 Hispanic Male',
       '85+ White Male', '85+ Black Male', '85+ AIAN Total', '85+ AIAN Male',
       '85+ Hispanic Female', '85+ Two or More Races Female'],
      dtype='object')


In [18]:
selected = data[['65-69 Asian Male', '70-74 White Male', '70-74 Hispanic Male',
       '80-84 Black Female', '80-84 AIAN Male', '80-84 Hispanic Male',
       '85+ White Male', '85+ Black Male', '85+ AIAN Total', '85+ AIAN Male',
       '85+ Hispanic Female', '85+ Two or More Races Female']]
selected.head(5)

,65-69 Asian Male,70-74 White Male,70-74 Hispanic Male,80-84 Black Female,80-84 AIAN Male,80-84 Hispanic Male,85+ White Male,85+ Black Male,85+ AIAN Total,85+ AIAN Male,85+ Hispanic Female,85+ Two or More Races Female
0,4.73,252.10,73.52,0.00,0.00,21.49,77.42,0.00,0.72,0.00,12.86,0.00
1,2.23,800.01,11.50,0.00,0.00,2.22,357.39,0.00,2.86,0.00,5.74,0.00
2,198.08,3940.75,252.68,7.72,16.73,105.36,1218.86,5.38,17.54,9.68,51.06,53.42
3,27.64,2189.33,143.19,1.70,7.70,35.51,841.28,2.70,4.21,0.00,40.53,12.51
4,31.38,2825.18,48.31,2.99,13.18,14.07,1215.94,2.16,20.05,4.42,17.60,9.87


In [19]:
y.head()

0     39
1     90
2    364
3    240
4    303
Name: DSHS nursing home beds, dtype: int64

In [20]:
selected_train, selected_valid, y_train, y_valid = train_test_split(selected, y, test_size = 0.5, random_state = 45)

In [21]:
rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
rf.fit(selected_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(selected_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(selected_valid, y_valid)))

R^2 Training Score: 0.96 
OOB Score: 0.76 
R^2 Validation Score: 0.10


In [22]:
rf.score(selected_valid, y_valid)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = selected_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

                              importance
70-74 White Male                0.154956
85+ Black Male                  0.132563
85+ White Male                  0.119711
80-84 AIAN Male                 0.117990
85+ Hispanic Female             0.105771
80-84 Black Female              0.101173
70-74 Hispanic Male             0.055692
80-84 Hispanic Male             0.048581
85+ Two or More Races Female    0.048173
65-69 Asian Male                0.046169
85+ AIAN Total                  0.039568
85+ AIAN Male                   0.029653


In [25]:
x.head()

,Total Population,Male,Female,White Total,White Male,White Female,Black Total,Black Male,Black Female,AIAN Total,...,85+ Asian Female,85+ NHOPI Total,85+ NHOPI Male,85+ NHOPI Female,85+ Hispanic Total,85+ Hispanic Male,85+ Hispanic Female,85+ Two or More Races Total,85+ Two or More Races Male,85+ Two or More Races Female
0,20150,10233.52,9916.48,18056,9185.27,8870.73,446,265.57,180.43,1109,...,1.32,0.0,0.0,0.0,25.03,12.16,12.86,1.30,1.30,0.00
1,22520,11100.48,11419.52,21087,10434.52,10652.48,144,81.98,62.02,356,...,2.02,2.1,0.0,2.1,11.97,6.23,5.74,6.55,6.55,0.00
2,201800,101396.07,100403.93,181938,91546.48,90391.52,3378,1845.42,1532.58,2504,...,36.25,0.0,0.0,0.0,114.56,63.50,51.06,61.50,8.08,53.42
3,78420,39199.90,39220.10,73391,36703.57,36687.43,510,300.71,209.29,1497,...,3.51,0.0,0.0,0.0,79.82,39.29,40.53,16.54,4.03,12.51
4,76010,37927.37,38082.63,65939,32773.48,33165.52,791,571.02,219.98,4242,...,16.03,0.0,0.0,0.0,27.72,10.13,17.60,22.90,13.03,9.87


In [41]:
a = data[['County','DSHS nursing home beds']]
a.head()

,County,DSHS nursing home beds
0,Adams County,39
1,Asotin County,90
2,Benton County,364
3,Chelan County,240
4,Clallam County,303
